In [1]:
from tqdm import tqdm
from datetime import date
import bs4, requests, json

In [13]:
home_url = r'http://thevoicemyanmar.com/news'

# get original page
soup = bs4.BeautifulSoup(requests.get(home_url).content, 'lxml')

# get trending links
main_menu_list = soup.select('.nav.navbar-nav')
main_menu_url_list = []
for atag in main_menu_list[0].find_all('a'):
    try:
        main_menu_url_list.append(atag['href'])
    except:
        pass
print(main_menu_url_list[0], len(main_menu_url_list))

http://thevoicemyanmar.com 50


In [55]:
art_url_list = []
article_list = []
for link in tqdm(main_menu_url_list):
    current_page = bs4.BeautifulSoup(requests.get(link).content, 'lxml')
    articles = current_page.select('article')
    for article_header in articles:
        try:
            art_url_list.append(article_header.select('.caption')[0].a['href'])
        except:
            pass

100%|██████████| 50/50 [00:25<00:00,  1.96it/s]


In [56]:
# Take Unique
art_url_list_unq = list(set(art_url_list))
print(len(art_url_list), len(art_url_list_unq))

762 666


In [69]:
N = 50 # we are only taking 50

article_list = []
for art_url in tqdm(art_url_list_unq[:N]):
    article = bs4.BeautifulSoup(requests.get(art_url).content, 'lxml')
    art_title = article.select('.caption.title_font')[0].text
    art_body = article.select('.description.zawgyi')
    article_list.append({
        'url': art_url,
        'title': art_title,
        'body': '\n'.join(para.text for para in art_body[0].find_all('div'))
    })

100%|██████████| 50/50 [00:24<00:00,  2.06it/s]


In [71]:
with open('data/thevoice_{}.json'.format(date.today()), 'w', encoding='utf-8') as f:
    json.dump(article_list, f, ensure_ascii=False, indent=4)